### Problem Set 9: NP Completeness and Reductions

csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2022.
<br>
Burton Rosenberg.
<br>
<br>last update: April 22, 2022


---

## Student name: Diep Vu

---

### Overview

This homework will program the polynomial time reduction from 3SAT to k-Vertex Cover. 

The input to the class with be an instance of 3SAT, and in each case, it will produce a graph and a value $k$, such that there is a $k$ cover if and only if the 3SAT is satisfiable and the satisfying assignement to the 3SAT will be extracted from the vertex cover.

The text book explains the reductions.

On the reduced graph, we will do an exhaustive search for the vertex cover, and then interpret that back as a truth assignement for the 3SAT. Thus solving the 3SAT by solving the vertex cover.

### Data representations

We shall represent a SAT instance as a list of clauses, where each clause is a list of pairs, where each pair is a variable name and the integer 0 if the variable appears alone, and 1 if the variable appears complemented. Note that this is the same represenntation as in the previous problem set. In our use, there shall be exactly three variables in any clause.

We will wrap a CNF inside an instance of a CNF object which adds a few useful methods.

A graph will be represented as an instance of a Graph class. The vertices and edges will be represented as lists.

It is not necessary to proceed in this manner, but I have suggested that the names of the vertices of the graph be derived from the names of the appearances of the variables, as they appear in the formula. That a variable can appear multiple times, each with a distinct name, can be handled by forming a pair, with the name in the form of the CNF an integer index, which continuously increments with each vertex.

<pre>
     vertex := ( (name: string, logic: {0,1}), index: integer )
</pre>

The classes Graph and CNF are provided for you.


### Project

The Reduce_3SAT_VCover class is instantiated with a CNF object. The _make_graph_ creates the Graph object with the vertices and edges in the reduction from 3SAT to vertex cover for the cnf formula given at the instantiation of the Reduce_3SAT_VCover instance.

I have suggested a format for the vertices of the graph. If the vertex widgets are created first, there will be an easy way to tell those vertices that are part of any vertex widget and those that are part of any clause widget, because of the index of the any vertex widget has a precise bound. This observation may be helpful.

I have suggested to break the graph construction into three parts,

- _make_variable_widget_: create the vertices and edges that are in pairs and represent values of the variables;
- _make_clause_widget_: create the vertices and edges that are in trianbles nd represent the clauses;
- _make_logic_edges_: create the edges that go between the variable widgets and the clause widgets.

Once built, and the $k$ computed, the function _find_cover_ finds the vertex cover using exhaustive search. There are several possibilities. It will all be exponential but how you do it might make a difference on whether the program can find covers in large graphs. The function _is_cover_ evaluates each proposed vertex list to see if it is a cover.

Finally, the function _solve_cnf_ takes the cover and extracts the associated True/False values of the variables, returning it as a dictionary suitable for tesing by the  _CNF.is_satisfied_ function.

I have the properties (instance variables in Java jargon) graph and cover in the Reduce_3SAT_VCover class, so that the methods can work step by step, filling these in with each call. This might help debugging.



In [1]:

class Graph:

    def __init__(self,vertices,edges):
        self.v = vertices
        self.e = edges
        
    def __repr__(self):
        # 
        # this code depends on the graph being of the suggested format
        # vertices are of the the form ((string,{0,1}),integer) where
        # string is the variable name, 1 means not the variable,and integer
        # is a unique variable index.
        #
        s = f"vertices:\n"
        for v in sorted(self.v,key=lambda x: x[1]):
            s += f'\t{v}\n'
        s += f"edges:\n"
        for e in sorted(self.e,key=lambda x: min(x[0][1],x[1][1])):
            if e[0][1]<=e[1][1]:
                s += f'\t{e[0]}:{e[1]}\n'
            else:
                s += f'\t{e[1]}:{e[0]}\n'
        return s
        
    def vertices(self):
        for v in self.v:
            yield v
    
    def edges(self):
        for e in self.e:
            yield e
            
    def has_edge(self,v,w):
        for e in self.edges():
            if e == (v,w) or e == (w,v):
                return True
        return False


class CNF: 
    
    def __init__(self,cnf):
        self.cnf = cnf
        self.var = self.collect_variables()

    def collect_variables(self):
        v = []
        for clause in self.cnf:
            for variable in clause:
                if variable[0] not in v:
                    v.append(variable[0])
        return v

    def variables(self):
        vi = self.var
        for v in vi:
            yield v
 
    def __repr__(self):

        def p_aux(d,s):
            f = False
            s += '('
            for v in d:
                if f:
                    s += ' OR '
                else:
                    f = True
                if v[1]==0:
                    s += f'{v[0]}'
                else:
                    s += f'~{v[0]}'
            s += f')'
            return s
            
        s = ''
        f = False
        for d in self.cnf:
            if f:
                s += ' AND '
            else:
                f = True
            s = p_aux(d,s)
        return s

    def is_satisfied_aux(self,disjunct,assignment):
        for v in disjunct:
            if v[0] not in assignment:
                return False
            if v[1]==0:
                if assignment[v[0]]:
                    return True
            else:
                if not assignment[v[0]]:
                    return True
        return False

    def is_satisfied(self, assignment):
        if assignment==None:
            return None
        for disjunct in self.cnf:
            if not self.is_satisfied_aux(disjunct,assignment):
                return False
        return True
    

# examples

fig_7_33 = [[('x1',0),('x1',0),('x2',0),],
            [('x1',1),('x2',1),('x2',1),],
            [('x1',1),('x2',0),('x2',0),],]

cnf = CNF(fig_7_33)
print(f'formula: {cnf}')
ev = {'x1':True,'x2':True}
print(f'\t{ev} -> {cnf.is_satisfied(ev)}')
ev = {'x1':False,'x2':True}
print(f'\t{ev} -> {cnf.is_satisfied(ev)}')

v = [(('x1',0),1),(('x1',1),2),(('x1',0),3),(('x1',0),4),(('x1',1),5)]
e = [
    ((('x1',0),1),(('x1',1),2)),
    ((('x1',0),3),(('x1',0),4)), 
    ((('x1',0),4),(('x1',1),5)), 
    ((('x1',1),5),(('x1',0),3)),
    ((('x1',0),1),(('x1',0),3)),
    ((('x1',0),1),(('x1',0),4)),
    ((('x1',1),2),(('x1',1),5)),
]
g = Graph(v,e)
print(g)

formula: (x1 OR x1 OR x2) AND (~x1 OR ~x2 OR ~x2) AND (~x1 OR x2 OR x2)
	{'x1': True, 'x2': True} -> False
	{'x1': False, 'x2': True} -> True
vertices:
	(('x1', 0), 1)
	(('x1', 1), 2)
	(('x1', 0), 3)
	(('x1', 0), 4)
	(('x1', 1), 5)
edges:
	(('x1', 0), 1):(('x1', 1), 2)
	(('x1', 0), 1):(('x1', 0), 3)
	(('x1', 0), 1):(('x1', 0), 4)
	(('x1', 1), 2):(('x1', 1), 5)
	(('x1', 0), 3):(('x1', 0), 4)
	(('x1', 0), 3):(('x1', 1), 5)
	(('x1', 0), 4):(('x1', 1), 5)



In [64]:
import itertools
import random

class Reduce_3SAT_VCover:
    
    def __init__(self,cnf):
        self.cnf = cnf
        self.index = 0
        self.graph = None
        self.k = 0
        self.cover = None


    def make_variable_widget(self):
        # make the sub-graph corresponding to all the variable widgets,
        v = []
        e = []
        
        # code
        
        for var in self.cnf.var:
            v.append( ((var, 0), self.index) )
            self.index += 1
            v.append( ((var, 1), self.index) )
            self.index += 1
            e.append( ( ((var, 0), self.index - 2), ((var, 1), self.index - 1) ) )

        return (v,e)

    def make_clause_widget(self):  
        # make the sub-graphs corresponding to all the clause widgets.
        v = []
        g = []
        
        # code 
        i = 0
                     
        for clause in self.cnf.cnf:
            for variable in clause:
                v.append( (variable, self.index) )
                self.index += 1
                
            g.append( (v[i],v[i + 1]) )
            g.append( (v[i],v[i + 2]) )       
            g.append( (v[i + 1],v[i + 2]) ) 
            i += 3
   
        return (v,g)    

    def make_logic_edges(self,v_variable,v_clause):
        # return a list of edges, the edges connecting the variable and clause widgets
        e = []
   
        for var in v_variable:
            for clause in v_clause:
                if (var[0][0] == clause[0][0] and var[0][1] == clause[0][1]):
                    e.append( (var,clause) )
            
        return e
    
    def make_graph(self):
        self.index = 0
        (v_variable,e_variable) = self.make_variable_widget()
        (v_clause,e_clause) = self.make_clause_widget()
        e_logic = self.make_logic_edges(v_variable,v_clause)
    
        self.graph = Graph(
            v_variable+v_clause,
            e_variable+e_clause+e_logic)

        #print(self.graph)
        
        assert (len(v_variable)%2)==0
        assert (len(v_clause)%3)==0
        self.k = len(v_variable)//2 + 2*len(v_clause)//3
        
    def is_cover(self,vc):

        # given a list of vertices vc, return True if a 
        # cover, False otherwise
        if (len(vc) == 0):
            return None
        
        for edge in self.graph.e:
            if edge[0] not in vc and edge[1] not in vc:
                return False
        
        return True   # using None here to note there is no implementation
 
    def find_cover(self):
        self.make_graph()
        temp =[]
        self.cover = []
    
        for j in list(itertools.combinations(self.graph.v, self.k)):
            temp.append(j)
            
        for covers in temp:
            if self.is_cover(covers) == True:
                self.cover.append(covers)
        
        if not self.cover:
            return None
                
        return self.cover
        
        
    def solve_cnf(self):
        
        #sol = {'x1':False, 'x2':False}  # an example
        sol = {}
        
        # given that there is a cover, convert it to a dictionary from variable
        # name to True/False, which is a satisfying assignment.
        # else return None
        sol_temp = []
        ind = 0

        if not self.cover:
            return None
        
        for v in self.cover[0]:
            if v[1] < 2 * len(self.cnf.var):
                sol_temp.append(v)
            
        for s in sol_temp:
            if (s[0][1] == 0):
                sol[s[0][0]] = True
            if (s[0][1] == 1):
                sol[s[0][0]] = False
            
                
        return sol  # using None here to note there is no implementation
 

In [65]:
def test(label, cnf):
    print(f'***\n*** {label}\n***')
    d = CNF(cnf)
    print(f'cnf:\t{d}')
    g = Reduce_3SAT_VCover(d)
    cover = g.find_cover()
    #cover = g.draft()
    print("My cover is the list of all possible k-vertex covers")
    print(f'cover:\t{cover}')
    sol = g.solve_cnf()
    print(f'sat:\t{sol}')
    print(f'check:\t{d.is_satisfied(sol)}')
    print()



ex_a = [[('x1',0),('x1',0),('x2',0),],
        [('x1',1),('x2',1),('x2',1),]]

ex_b = [[('x1',0),('x1',0),('x1',0),],
        [('x1',1),('x1',1),('x1',1),]]

fig_7_33 = [[('x1',0),('x1',0),('x2',0),],
            [('x1',1),('x2',1),('x2',1),],
            [('x1',1),('x2',0),('x2',0),],]
  
test('Example A', ex_a)
test('Example B', ex_b)
test('Figure 7.33', fig_7_33)        


***
*** Example A
***
cnf:	(x1 OR x1 OR x2) AND (~x1 OR ~x2 OR ~x2)
My cover is the list of all possible k-vertex covers
cover:	[((('x1', 0), 0), (('x2', 1), 3), (('x1', 0), 4), (('x2', 0), 6), (('x1', 1), 7), (('x2', 1), 8)), ((('x1', 0), 0), (('x2', 1), 3), (('x1', 0), 4), (('x2', 0), 6), (('x1', 1), 7), (('x2', 1), 9)), ((('x1', 0), 0), (('x2', 1), 3), (('x1', 0), 5), (('x2', 0), 6), (('x1', 1), 7), (('x2', 1), 8)), ((('x1', 0), 0), (('x2', 1), 3), (('x1', 0), 5), (('x2', 0), 6), (('x1', 1), 7), (('x2', 1), 9)), ((('x1', 1), 1), (('x2', 0), 2), (('x1', 0), 4), (('x1', 0), 5), (('x2', 1), 8), (('x2', 1), 9))]
sat:	{'x1': True, 'x2': False}
check:	True

***
*** Example B
***
cnf:	(x1 OR x1 OR x1) AND (~x1 OR ~x1 OR ~x1)
My cover is the list of all possible k-vertex covers
cover:	None
sat:	None
check:	None

***
*** Figure 7.33
***
cnf:	(x1 OR x1 OR x2) AND (~x1 OR ~x2 OR ~x2) AND (~x1 OR x2 OR x2)
My cover is the list of all possible k-vertex covers
cover:	[((('x1', 1), 1), (('x2', 0),

def find_cover(self):
        
        #cover = [(('x1',1),0), (('x2',1),1)]  # an example
        
        # use exhaustive search to find and return a VC as a list of vertices
        # if there is no cover, return None
        
        # Initialize all vertices as not visites
        self.make_graph()
        self.cover = []
        numEdge = [0] * (self.index)
                
        for edge in self.graph.e:
            numEdge[edge[0][1]] += 1
            numEdge[edge[1][1]] += 1
            
        index_of_max = numEdge.index(max(numEdge))
        cover_copy = []
        edgeCopy = self.graph.e.copy()
        total_edge = len(edgeCopy)
        
        #print(total_edge)
        #while total_edge > 0:
            #for i in range(self.index):
                #if (i == max(index))   # i is vertex with max number of edges connected to it
                    #index_of_max = numEdge.index(i)
                    #curr_vertex = self.graph.v[index_of_max]
        temp1 = []
                    
                    
        while total_edge > 0:
            temp1 = []
            for i in range(self.index):
                if (numEdge[i] == max(numEdge)):
                    temp1.append(i)
                    
            #index_of_max = numEdge.index(max(numEdge))
            
            index_of_max = random.choice(temp1)
            
            curr_vertex = self.graph.v[index_of_max]
            
            numEdge[index_of_max] = 0
            edge_copy_copy = edgeCopy.copy()
            
            self.cover.append(curr_vertex)
            for e in edgeCopy:
                if e[0] == curr_vertex:
                    edge_copy_copy.remove((e[0],e[1]))
                    numEdge[e[1][1]] -= 1
                    total_edge -= 1
                    
                elif e[1] == curr_vertex:
                    edge_copy_copy.remove((e[0],e[1]))
                    numEdge[e[0][1]] -= 1
                    total_edge -= 1
                    
            edgeCopy = edge_copy_copy.copy()
        
                
            #print(total_edge)
            
        #print(numEdge)
    
        #print(str(edge[0][1]) + " " + str(edge[1][1])
        
        #for vertex in self.cover:
            #print(vertex)
            
        #print(len(self.cover))
        
        #print(self.k)
        
        return None  # using None here to note there is no implementation